1. Importar librerías

In [ ]:
# PEP8 es una guía de estilo para escribir código Python. (https://www.python.org/dev/peps/pep-0008/)

# Librerías estándar de python
import os
import tarfile
import urllib.request as request
import pandas as pd
# Librerías de terceros
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, r2_score

# Librerías locales
# from mi_clase import AwsStorage

%matplotlib inline

In [1]:
import os

# Carpeta donde están los datos
DATA_DIR = os.path.join("data")

# Ruta completa al archivo
INGRESOS_PATH = os.path.join(DATA_DIR, "ingresos.txt")


In [2]:
print(INGRESOS_PATH)

data\ingresos.txt


3. Extracción de los datos

In [ ]:
import os

# Ruta a la carpeta de datos
DATA_DIR = os.path.join("data")
# Ruta completa al archivo
INGRESOS_PATH = os.path.join(DATA_DIR, "ingresos.txt")

def cargar_datos(path: str = INGRESOS_PATH) -> pd.DataFrame:
    # Verifica que el archivo exista
    if not os.path.isfile(path):
        raise FileNotFoundError(f"Archivo no encontrado en: {path}")
    
    # Carga el archivo CSV y lo devuelve como DataFrame
    return pd.read_csv(path)